**Lista de Exercícios N2**

Tema: Grafos e Algoritmos de Busca (Dijkstra, A*, em-ordem, pré-ordem e pós-ordem)

Disciplina: Inteligência Artificial\
Ambiente: Google Colab\
Entrega: via repositório individual no GitHub

**Nome completo:**\
**Matricula:**

**Regras Gerais**

- Trabalho individual.

- Linguagem: Python 3 (Google Colab).

- É permitido usar: heapq, numpy, matplotlib, dataclasses.

- Proibido usar funções prontas de shortest path (networkx.shortest_path, scipy.sparse.csgraph.dijkstra, etc.).

O Notebook (.ipynb) deve conter:

- Identificação (nome, turma, link do GitHub)

- Código, testes e reflexões

- Seções organizadas conforme o roteiro abaixo.

**Parte A — Dijkstra (Caminho Mínimo em Grafos Ponderados Positivos)**

> Imagine que você está projetando um sistema de navegação para ambulâncias em uma cidade. Cada interseção é representada como um nó e cada rua como uma aresta ponderada com o tempo médio de deslocamento. Você precisa encontrar a rota mais rápida entre o hospital e o local de atendimento, garantindo que o caminho tenha custo mínimo e seja correto mesmo em grandes redes urbanas.

**Atividades**

- Implemente o algoritmo de Dijkstra, retornando o custo mínimo (dist) e os predecessores (parent).

- Crie uma função reconstruct_path(parent, target) que reconstrua o trajeto.

Teste o algoritmo em um grafo de exemplo.

**Explique (Questões Discursivas):**

1. Por que Dijkstra exige arestas não negativas?

2. Qual a complexidade do algoritmo com lista de adjacência e heapq?

💡 Dica: Compare seu resultado com um mapa simples — se mudar o peso de uma rua, a rota muda?

In [ ]:
# Insira seu código aqui

**Parte B — A-Star (Busca Informada com Heurística Admissível)**

> Agora, considere um robô autônomo que deve se deslocar por um labirinto 2D, evitando obstáculos e chegando ao destino no menor tempo possível.
Diferente do Dijkstra, o robô pode usar uma heurística (como a distância ao alvo) para priorizar rotas promissoras, economizando tempo de busca.

**Atividades**

1. Gere um grid 20x20 com ~15% de obstáculos aleatórios.

2. Implemente a função heuristic(a, b) (distância Manhattan).

3. Desenvolva o algoritmo a_star(grid, start, goal, h) e teste-o.

**Explique (Questões Discursivas):**

1. A* vs Dijkstra: qual expande menos nós?

2. Por que a heurística Manhattan é admissível nesse caso?

💡 Cenário real: O A* é amplamente usado em robôs aspiradores, drones e jogos. Seu desafio é aplicar o mesmo raciocínio.

In [ ]:
# Insira seu código aqui

**Parte C — Árvores Binárias e Percursos (DFS em-ordem, pré-ordem e pós-ordem)**

> Você está desenvolvendo um sistema de recomendação que organiza produtos em uma árvore binária de busca (BST), conforme o preço. Cada nó é um produto e a travessia da árvore pode ser usada para: 1. Ordenar produtos (em-ordem); 2. Clonar a estrutura (pré-ordem); 3. Calcular totais ou liberar memória (pós-ordem);

**Atividades**

1. Crie uma BST com os valores: [50, 30, 70, 20, 40, 60, 80, 35, 45].

Implemente os percursos:

1. in_order(root)

2. pre_order(root)

3. post_order(root)

Teste se as saídas correspondem às travessias esperadas.

**Explique (Questões Discursivas):**

1. Em que situação cada tipo de percurso é mais indicado?

In [2]:
## Insira seu código aqui

**Parte D — Reflexões (Respostas Curtas)**

Responda de forma argumentativa (5–10 linhas cada):

1. Quando não é vantajoso usar A*, mesmo tendo uma heurística?

2. Diferencie corretude e otimalidade nos algoritmos estudados.

3. Dê um exemplo do mundo real onde cada tipo de percurso (em, pré, pós) é essencial.

4. Como heurísticas inconsistentes podem afetar o resultado do A*?

💬 Sugestão: use exemplos de mapas, jogos, sistemas de busca ou árvores sintáticas.

Insira suas respostas aqui

**Entrega**

Crie um repositório público chamado ia-grafos-seu-nome.

Envie para o repositório o arquivo ia_grafos_buscas.ipynb.

Submeta o link do repositório no ambiente da disciplina **Portal Digital Fametro.**.

**Integridade Acadêmica**

1. O trabalho é individual.
2. Discussões conceituais são permitidas, mas o código deve ser inteiramente autoral.
3. Verificações de similaridade serão aplicadas a todas as submissões.
4. Busque e estude os algoritmos através de pesquisas na internet, livros, slides da disciplina.